## developing infinite alleles

- re-write each function
- write a test suite for each function

In [92]:
import os
import sys

import numpy as np
import pandas as pd
import random

import allel

from lib.sequencing import *

import matplotlib.pyplot as plt
import seaborn as sns
%matplotline line

UsageError: Line magic function `%matplotline` not found.


## new mutation function

In [2]:
def evolve_host(hh, ti, theta=0.0, drift_rate=0.0, nsnps=0, back_mutation=False):
    """
    Evolve the parasite genomes of a host forward `ti` days,
    simulating drift and mutation, and optionally
    allowing for back mutation
    
    Implements a continuous-time Moran model with
    mutation

    Parameters
        hh: ndarray, shape (npv, nsnps)
            Array containing parasite genomes for single host.
        ti: float|
            Amount of time to simulate forward, measured in days.
            I.e. difference between present time and last update.
        theta: float
            Mutation probability per base per drift event.
        drift_rate: float
            Expected number of drift events per day.
        nsnps: int
            Number of SNPs per parasite genome.
        back_mutation: bool
            Allow for back mutation?
    Returns
        hh: ndarray, shape (npv, nsnps)
            Array containing parasite genomes for a single genome,
            after evolving.
    """
    
    nreps = np.random.poisson(ti * drift_rate)  # number of reproductions
    if nreps > 0:
        # Prepare
        s = np.random.choice(len(hh), size=(nreps, 2))  # strains 
        m = np.random.uniform(size=nreps) < theta * nsnps  # mutations
        m_ix = np.random.choice(nsnps, m.sum())  # mutation sites
        # Sequentially run through events
        j = 0
        for i in np.arange(nreps):
            if m[i]:  # mutation
                if back_mutation:
                    hh[s[i, 0], m_ix[j]] = 3 - hh[s[i, 0], m_ix[j]]
                else:
                    hh[s[i, 0], m_ix[j]] = 2
                j += 1
            else:  # drift
                hh[s[i, 1]] = hh[s[i, 0]]
    
    return hh

### diagnostics
- The key conclusion is that `random` is faster for individual calls, and `np` is faster if you want to do many calls

In [39]:
%timeit random.random()

57.2 ns ± 1.19 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [40]:
%timeit np.random.uniform()

1.58 µs ± 14.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [41]:
(1.58*10**-6)/(57.2*10**-9)

27.622377622377616

In [61]:
%timeit np.random.uniform(0, 1, 10)

2.56 µs ± 46.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [62]:
%timeit [random.random() for _ in range(10)]

1.1 µs ± 11.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### introduce infinite alleles mutation

In [63]:
def evolve_host_infty(hh, ti, theta=0.0, drift_rate=0.0, nsnps=0, back_mutation=False):
    """
    Evolve the parasite genomes of a host forward `ti` days,
    simulating drift and mutation, and optionally
    allowing for back mutation
    
    Implements a continuous-time Moran model with
    mutation

    Parameters
        hh: ndarray, shape (npv, nsnps)
            Array containing parasite genomes for single host.
        ti: float
            Amount of time to simulate forward, measured in days.
            I.e. difference between present time and last update.
        theta: float
            Mutation probability per base per drift event.
        drift_rate: float
            Expected number of drift events per day.
        nsnps: int
            Number of SNPs per parasite genome.
        back_mutation: bool
            Allow for back mutation?
    Returns
        hh: ndarray, shape (npv, nsnps)
            Array containing parasite genomes for a single genome,
            after evolving.
    """
    
    nreps = np.random.poisson(ti * drift_rate)  # number of reproductions
    if nreps > 0:
        # Prepare
        s = np.random.choice(len(hh), size=(nreps, 2))  # select strains
        m = np.random.uniform(size=nreps) < theta * nsnps  # mutate?
        n_m = m.sum()
        m_ix = np.random.choice(nsnps, n_m)  # if so, what site
        m_allele = np.random.uniform(0, 1, n_m) # generate alleles
        # Sequentially run through events
        j = 0
        for i in np.arange(nreps):
            if m[i]:  # mutation
                hh[s[i, 0], m_ix[j]] = m_allele[j]
                j += 1
            else:  # drift
                hh[s[i, 1]] = hh[s[i, 0]]
    
    return hh

In [207]:
def evolve_host_infinite_alleles(hh, ti, theta, drift_rate, nsnps):
    """
    Evolve a host genome using the infinite alleles model
    """
    nreps = np.random.poisson(ti * drift_rate)  # number of reproductions
    
    if nreps > 0:
        # For each reproduction...
        selected = np.random.choice(len(hh), size=(nreps, 2))  # select strains
        mutations = np.random.uniform(size=nreps) < theta * nsnps  # determine if mutation
        n_mutations = mutations.sum()
        mutation_position = np.random.choice(nsnps, n_mutations)  # select mutation position
        mutation_allele = np.random.uniform(0, 1, n_mutations)  # generate mutation allele
        
        # Sequentially simulate reproductions
        j = 0
        for i, mutation in enumerate(mutations):
            if mutation:
                hh[selected[i, 0], mutation_position[j]] = mutation_allele[j]
                j += 1
            else:  # drift
                hh[selected[i, 1]] = hh[selected[i, 0]]
        
        return hh

In [324]:
drift_h = 1.0
theta_h = 0.0002
nsnps = 1000

In [232]:
hh = np.zeros((10, 1000))
hh = evolve_host_infinite_alleles(hh, ti=10**5, theta=theta_h, drift_rate=drift_h, nsnps=nsnps)
hh

array([[0.80101321, 0.55240734, 0.        , ..., 0.52379288, 0.82107478,
        0.2729045 ],
       [0.80101321, 0.55240734, 0.        , ..., 0.52379288, 0.82107478,
        0.2729045 ],
       [0.80101321, 0.55240734, 0.        , ..., 0.52379288, 0.82107478,
        0.2729045 ],
       ...,
       [0.80101321, 0.55240734, 0.        , ..., 0.52379288, 0.82107478,
        0.2729045 ],
       [0.80101321, 0.55240734, 0.        , ..., 0.52379288, 0.82107478,
        0.2729045 ],
       [0.80101321, 0.55240734, 0.        , ..., 0.52379288, 0.82107478,
        0.2729045 ]])

In [213]:
%timeit evolve_host(hh, ti=10**3, theta=theta_h, drift_rate=drift_h, nsnps=nsnps)

1 ms ± 13.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [211]:
%timeit evolve_host_infty(hh, ti=10**3, theta=theta_h, drift_rate=drift_h, nsnps=nsnps)

1.02 ms ± 13.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [212]:
%timeit evolve_host_infinite_alleles(hh, ti=10**3, theta=theta_h, drift_rate=drift_h, nsnps=nsnps)

896 µs ± 3.65 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### Tests
- Look at some of Jerome Kheller's models for what would be good as far as testing goes
- Some ideas are:
    - Confirm behaviour matches what is expected from Moran without mutation (fixation times)
    - Confirm allele counts follows Ewen's Sampling Formula
    - Confirm that as we increase the mutation rate, we get a proportional increase in the number of mutations

## Computing genetic diversity statistics in the infinite alleles model

### Sequence DNA
- The first thing we need to do is "extract genomes"
- These are the unique haplotypes present within the infection

In [229]:
def sequence_dna_infinite_alleles(hh, nsnps, detection_threshold=None):
    """
    Sequence parasite genomes within a single host
    reporting `k`, the number of strains present
    and `seqs`, the sequences themselves

    Note that identical repeated parasite genomes are
    reported only once by `sequence_dna()`

    Parameters
        hh: ndarray, shape (nph, nsnps)
            Array containing parasite genomes for single host.
        detection_threshold: None or float
            A haplotype is only sequenced if > `detection_threshold`
            of its sites are heterozygous, when compared to *all*
            other collected genomes.

    Returns
        k: int
            Number of unique parasite genomes within the
            sequenced host.
        seqs: ndarray, shape(k, nsnps)
            Sequenced parasite genomes.
    """
    seqs = np.vstack(list({tuple(row) for row in hh}))
    k = seqs.shape[0]
    if k > 1 and detection_threshold is not None:  # now we check for enough differences
        keep_indxs = [0]
        for j in np.arange(1, k):
            ndiffs = (seqs[keep_indxs, :] != seqs[j]).sum(1)
            if (ndiffs / nsnps >= detection_threshold).all():
                keep_indxs.append(j)
        seqs = seqs[keep_indxs]
        k = seqs.shape[0]
    return k, seqs

In [238]:
k, seqs = sequence_dna_infinite_alleles(hh, nsnps)

#### Tests
- If they are all identical, return one
- If all different, return ten
- Test outputs to ensure sufficiently different as expected
    - e.g. run a bunch of random sequences through and ensure they are always more different than the threshold
- what about bad inputs?

In [246]:
(seqs[0, :] == seqs[1:, :]).sum(1) / nsnps

array([0.991, 0.994, 0.991, 0.992])

### Compute the allele frequency spectrum
- The fastest implementation will be a vectorised one
- For now all I have is a loop
- The input for this is a `haplotype` array, where the rows are snps and the columns are samples
    - So we need to transpose the sequences

In [269]:
genomes = seqs.T
genomes.shape

(1000, 5)

In [264]:
def get_allele_counts(genomes):
    """
    Generate an allele count array
    for a number of genomes
    """
    nsnps, ngenomes = genomes.shape
    ac = np.zeros((nsnps, ngenomes), 'int16')  # the maximum possible size
    for i in np.arange(nsnps):
        counts = np.unique(genomes[i], return_counts=True)[1]
        n = len(counts)
        ac[i, :n] = counts
    ac = ac[:, ac.sum(0) > 0]  # remove columns with no alleles
    return allel.AlleleCountsArray(ac)

In [265]:
ac = get_allele_counts(genomes)
ac

<AlleleCountsArray shape=(1000, 2) dtype=int16>
5 0
5 0
5 0
...
5 0
5 0
5 0

In [266]:
%timeit get_allele_counts(genomes)

13.7 ms ± 217 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Tests
- Test an example where all the alleles are reference
- Test an example with only one strain
- Test an example with the *maximum* number of possible alleles
- Test a small genome where you know the number of unique alleles in each row

In [270]:
gs = np.array([[0, 0, 1], [0, 1, 2], [0, 0, 0]])

In [271]:
get_allele_counts(gs)

,0,1,2
0,2,1,0
1,1,1,1
2,3,0,0


## Compute the classic genetic diversity statistics using `scikit-allel`
- Need to think about what makes sense here for an infinite alleles model
- Like, what do we mean by number of singletons?
    - I would imagine we mean number of instances of alleles that occur only one time *in our sample*

In [316]:
# For this we need variant "positions"
# For now, just contiguous array of integers
pos = np.arange(nsnps) + 1
pos[:10]

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [295]:
u = [i for i, row in enumerate(genomes) if len(np.unique(row)) > 1]
print(len(u))
u

10


[135, 248, 295, 431, 539, 629, 638, 652, 670, 677]

In [303]:
ac

<AlleleCountsArray shape=(1000, 2) dtype=int16>
5 0
5 0
5 0
...
5 0
5 0
5 0

In [301]:
genomes[u]

array([[0.00597416, 0.00597416, 0.00597416, 0.00597416, 0.03525214],
       [0.22090046, 0.92642778, 0.92642778, 0.92642778, 0.92642778],
       [0.85995135, 0.97084456, 0.97084456, 0.97084456, 0.97084456],
       [0.22074267, 0.22074267, 0.25949568, 0.25949568, 0.25949568],
       [0.57741349, 0.57741349, 0.57741349, 0.74165632, 0.74165632],
       [0.25381128, 0.6789414 , 0.6789414 , 0.6789414 , 0.6789414 ],
       [0.0515016 , 0.0515016 , 0.0515016 , 0.0515016 , 0.64625313],
       [0.44317385, 0.44317385, 0.44317385, 0.44317385, 0.93819445],
       [0.        , 0.        , 0.        , 0.        , 0.88640432],
       [0.17697397, 0.35769346, 0.35769346, 0.35769346, 0.35769346]])

In [296]:
genomes[248]

array([0.22090046, 0.92642778, 0.92642778, 0.92642778, 0.92642778])

In [312]:
# Number of variants (what is this?)
n_variants = ac.count_variant()

# Compute number of segregating sites
n_segregating = ac.count_segregating()

# Compute singletons
n_singletons = (ac == 1).any(1).sum()
print("Variants: %d" % n_variants)
print("Singletons: %d" % n_singletons)
print("Segregating: %d" % n_segregating)

Variants: 10
Singletons: 8
Segregating: 10


In [318]:
pi = allel.stats.diversity.sequence_diversity(pos, ac)
theta = allel.stats.diversity.watterson_theta(pos, ac)
tajd = allel.stats.diversity.tajima_d(ac)  # <4 samples, throws warning (NOT exception)

In [322]:
theta  # recall, in this case we just have a neutral moran model, what do we expect here?

0.0048000000000000004

In [323]:
theta  # don't really understand what I should expect here

0.0048000000000000004

In [326]:
theta_h * 10

0.002

In [327]:
# Number of variant sites doesn't make sense anymore

In [328]:
def calc_diversity_stats_infinite_alleles(pos, ac, verbose=False):
    """
    Calculate a suite of standard genetic diversity
    statistics given SNP positions and allele counts,
    for a set of parasite genomes

    Parameters
        ac: AlleleCountArray, shape (nsnps, 2)
            Allele counts computed from `hap`.
        pos: ndarray, shape (nsnps)
            The position, as an integer, of each SNP
            in the parasite genome.
    Returns
        div_stats: dict
            Dictionary of genetic diversity statistics.
    """
    _, n_allele_types = ac.shape
    if n_allele_types == 1:  # population clonal, f(x)'s would fail
        n_segregating = 0
        n_singletons = 0
        statement = "  Sample is clonal."
    else:
        n_segregating = ac.count_segregating()
        n_singletons = (ac == 1).any(1).sum()
        statement = "  Sample is not clonal."

    pi = allel.stats.diversity.sequence_diversity(pos, ac)
    theta = allel.stats.diversity.watterson_theta(pos, ac)
    tajd = allel.stats.diversity.tajima_d(ac)  # <4 samples, throws warning (NOT exception)

    if verbose:
        print("Diversity Metrics")
        print(statement)
        print("  No. Segregating Sites:", n_segregating)
        print("  No. Singletons:", n_singletons)
        print("  Nucleotide Diversity (pi):", pi)
        print("  Watterson's Theta:", theta)
        print("  Tajima's D:", tajd)

    div_stats = {"n_segregating": n_segregating,
                 "n_singletons": n_singletons,
                 "pi": pi,
                 "theta": theta,
                 "tajd": tajd}

    return div_stats

In [330]:
calc_diversity_stats_infinite_alleles(pos, ac, verbose=True)

Diversity Metrics
  Sample is not clonal.
  No. Segregating Sites: 10
  No. Singletons: 8
  Nucleotide Diversity (pi): 0.0044
  Watterson's Theta: 0.0048000000000000004
  Tajima's D: -0.5963326941320872


{'n_segregating': 10,
 'n_singletons': 8,
 'pi': 0.0044,
 'theta': 0.0048000000000000004,
 'tajd': -0.5963326941320872}

#### Tests
- Well, basic tests that the functions are computing correctly
- Throw clonal population, and a non-clonal population with known statistic values
    - Including known number of segregating sites, known number of singletons, &c.

In [134]:
hh

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [133]:
%timeit np.vstack(list({tuple(row) for row in hh}))

976 µs ± 18 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [108]:
?np.unique

In [93]:
hh.sum()

22.577056625593496

In [94]:
hh

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [98]:
genomes = np.vstack(list({tuple(row) for row in hh}))

In [99]:
gen_allel_datastructs(genomes)

TypeError: bad dtype kind: expected on of ('u', 'i'); found f

In [97]:
hh.shape

(10, 1000)

In [81]:
print(hh[:5, :10])

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.33387161
  0.         0.         0.         0.29349463]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.34023498 0.        ]
 [0.         0.11440944 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.63656763 0.         0.        ]]
